# Multi-Agent Support
<img src="https://github.com/AgentOps-AI/agentops/blob/b4aac2d4b9fb16d6aa0a25aa9018210a94f1bef2/docs/logo/multion_integration.png?raw=true" width="250px" style="max-width: 100%; height: auto;"/>
    
AgentOps supports tracking multiple agents in a single session. With AgentOps, agent actions as well as MultiOn browse events will get tracked. MultiOn browse events automatically trace screenshots as well.

This is an example implementation of tracking events from two separate agents


In [ ]:
import agentops
from agentops.agent import track_agent
from dotenv import load_dotenv
import os
from openai import OpenAI
import logging

from IPython.display import display, Markdown

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', "<your_openai_key>")
AGENTOPS_API_KEY = os.getenv('AGENTOPS_API_KEY', "<your_agentops_key>")
logging.basicConfig(level=logging.DEBUG) # this will let us see that calls are assigned to an agent

In [ ]:
agentops.init(AGENTOPS_API_KEY)
openai_client = OpenAI(api_key = OPENAI_API_KEY )

Now lets create a few agents!

In [ ]:
@track_agent(name='qa')
class QaAgent:
    def completion(self, prompt: str):
        res = openai_client.chat.completions.create(model='gpt-3.5-turbo', 
                                                    messages=[{"role": "system", 
                                                               "content": "You are a qa engineer and only output python code, no markdown tags."},
                                                              {"role": "user", "content": prompt}],
                                                    temperature=0.5)
        return res.choices[0].message.content
        
@track_agent(name='engineer')
class EngineerAgent:
    def completion(self, prompt: str):
        res = openai_client.chat.completions.create(model='gpt-3.5-turbo',
                                                    messages=[{"role": "system", 
                                                               "content": "You are a software engineer and only output python code, no markdown tags."},
                                                              {"role": "user", "content": prompt}], 
                                                    temperature=0.5)
        return res.choices[0].message.content

In [ ]:
qa = QaAgent()
engineer = EngineerAgent()

Now we have our agents and we tagged them with the `@track_agent` decorator. Any LLM calls that go through this class will now be tagged as agent calls in AgentOps.

Lets use these agents!

In [ ]:
generated_func = engineer.completion("Write a python function that accepts two numbers and multiplies them together, then divides by two. No example.")

In [ ]:
display(Markdown('```python\n' + generated_func + '\n```'))

In [ ]:
generated_test = qa.completion("Write a python unit test that test the following function: \n " + generated_func)

In [ ]:
display(Markdown('```python\n' + generated_test + '\n```'))

Perfect! It generated the code as expected, and in the DEBUG logs, you can see that the calls were made by agents named "engineer" and "qa"!

Lets verify one more thing! If we make an LLM call outside of the context of a tracked agent, we want to make sure it gets assigned to the Default Agent.

In [ ]:
res = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": "You are not a tracked agent"},
    {"role": "user", "content": "Say hello"}]
)
res.choices[0].message.content

You'll notice that we didn't log an agent name, so the AgentOps backend will assign it to the Default Agent for the session!